**Mount Google Drive to Collaboratory:**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Imports**

In [ ]:
import pandas as pd

**Read Datasets**

In [ ]:
past=pd.read_csv("/content/gdrive/MyDrive/DAV_Paper/sdg_index_2000-2022.csv")
past.head(5)

,country_code,country,year,sdg_index_score,goal_1_score,goal_2_score,goal_3_score,goal_4_score,goal_5_score,goal_6_score,...,goal_8_score,goal_9_score,goal_10_score,goal_11_score,goal_12_score,goal_13_score,goal_14_score,goal_15_score,goal_16_score,goal_17_score
0,AFG,Afghanistan,2000,36.0,28.8,27.3,19.2,1.6,20.8,32.4,...,38.5,5.2,0.0,25.8,94.7,99.4,0.0,51.9,39.2,34.2
1,AFG,Afghanistan,2001,36.3,28.8,30.6,19.4,1.6,20.8,32.4,...,38.5,5.2,0.0,25.8,94.5,99.4,0.0,51.9,39.2,34.2
2,AFG,Afghanistan,2002,36.3,28.8,30.7,19.7,1.6,20.8,32.7,...,38.4,5.2,0.0,26.1,94.1,99.4,0.0,51.8,39.2,34.2
3,AFG,Afghanistan,2003,36.7,28.8,32.5,19.9,1.6,20.8,33.0,...,38.4,5.2,0.0,26.5,94.4,99.4,0.0,51.8,39.2,34.2
4,AFG,Afghanistan,2004,37.1,28.8,32.1,21.1,1.6,20.8,33.3,...,38.5,5.2,0.0,26.8,94.8,99.4,0.0,51.8,39.2,34.2


In [ ]:
current=pd.read_csv("/content/gdrive/MyDrive/DAV_Paper/sustainable_development_report_2023.csv")
current.head()

,country_code,country,region,overall_score,goal_1_score,goal_2_score,goal_3_score,goal_4_score,goal_5_score,goal_6_score,...,goal_8_score,goal_9_score,goal_10_score,goal_11_score,goal_12_score,goal_13_score,goal_14_score,goal_15_score,goal_16_score,goal_17_score
0,FIN,Finland,OECD,86.760595,99.5750,60.886750,95.386385,97.169333,92.11125,94.3276,...,86.789000,95.995714,98.4685,91.233750,60.059571,68.793667,87.928000,85.0700,92.521091,75.60100
1,SWE,Sweden,OECD,85.981397,98.8885,63.074125,96.904000,99.761667,91.44025,95.0576,...,84.966429,97.586286,94.9650,90.389250,56.830571,70.031000,69.348667,80.1882,88.508455,85.77025
2,DNK,Denmark,OECD,85.683637,99.2155,71.025250,95.398500,99.339667,86.99800,90.7316,...,87.562429,96.984857,98.1560,93.038500,44.571714,60.780667,76.303333,92.7924,93.844909,82.14800
3,DEU,Germany,OECD,83.358447,99.5105,72.366000,93.039357,97.162667,81.92025,88.4434,...,86.967286,95.788429,88.1470,90.096500,55.412857,64.002000,73.996000,79.2318,89.457545,84.39025
4,AUT,Austria,OECD,82.280189,99.4510,73.067500,92.468000,97.914333,84.57925,92.1636,...,83.274143,96.982143,94.6345,92.473667,49.623286,57.332000,NaN,73.5836,87.911455,71.13025


**Data Preprocessing**

In [ ]:
# For past data separate yearly data concerning countries and yearly data about regions
unique_country_codes = set(current['country_code'])
mask = past['country_code'].isin(unique_country_codes)

past_countries = past[mask]
past_regions = past[~mask]

# Make copy of current trends data
current_copy = current.copy()

# Change a field name in dataset about current trends to simplify merging process
current_copy.rename(columns={'overall_score': 'sdg_index_score'}, inplace=True)

# Add year field to dataset about current trends to simplify merging process
current_copy['year'] = 2023

# Add region field to past trends data
country_region_mapping = current[['country', 'region']].drop_duplicates()
past_countries = past_countries.merge(country_region_mapping, on='country', how='left')

# Merge the two datasets
merged_data = pd.concat([past_countries, current_copy], ignore_index=True)

# Rearrange columns to match the desired order
merged_data = merged_data[['country_code', 'country', 'year', 'region', 'sdg_index_score',
                       'goal_1_score', 'goal_2_score', 'goal_3_score', 'goal_4_score',
                       'goal_5_score', 'goal_6_score', 'goal_7_score', 'goal_8_score',
                       'goal_9_score', 'goal_10_score', 'goal_11_score', 'goal_12_score',
                       'goal_13_score', 'goal_14_score', 'goal_15_score', 'goal_16_score',
                       'goal_17_score']]

# Sorting DataFrame by 'country' and 'year' to ensure the order
merged_data.sort_values(by=['country', 'year'], inplace=True)

# Fill null values and rearrange indexing
merged_data.fillna(0, inplace=True)
merged_data.reset_index(drop=True, inplace=True)

merged_data.to_csv('final_data.csv', index=False)